In [ ]:
!pip install afinn
!python -m textblob.download_corpora
!pip install -U textblob
!pip install vaderSentiment
!pip install pyemd
!pip install unidecode
!pip install pandas
!pip install gensim
!pip install sklearn
!pip install matplotlib

Finished.


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


Requirement already up-to-date: textblob in c:\users\user\appdata\roaming\python36\lib\site-packages (0.15.3)

ERROR: Cannot uninstall 'certifi'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.



  Using cached certifi-2020.6.20-py2.py3-none-any.whl (156 kB)
  Attempting uninstall: certifi
    Found existing installation: certifi 2018.4.16


In [ ]:
!pip install numpy

In [ ]:
!pip install -U numpy

Requirement already up-to-date: numpy in c:\users\user\appdata\roaming\python36\lib\site-packages (1.19.1)


In [ ]:
import re
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
from unidecode import unidecode
import pandas as pd
import bz2
import gensim
import warnings
import numpy as np
from gensim.models import word2vec
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

warnings.filterwarnings('ignore')
tqdm_notebook.pandas()


c:\users\user\appdata\roaming\python36\lib\site-packages\ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  app.launch_new_instance()


# packages in environment at c:\users\user\appdata\roaming\python36:
#
# Name                    Version                   Build  Channel
matplotlib                3.3.1                         0  
matplotlib-base           3.3.1            py36h4c87cce_0  

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Carregando os embeddings

Aqui vamos utilizar os embeddings para realizar as seguintes atividades:

- análise de simlaridade
- classificação de documentos

<b> Carregue os embeddings treinados, como vimos na Aula 2. É o mesmo arquivo que iremos utilizar</b>

Link: https://drive.google.com/open?id=1zI8pGfbUHuU_0wY_FV4tD6w6ZCUJTQbh

In [ ]:
newfilepath = "embedding_wiki_100d_pt.txt"
filepath = "ptwiki_20180420_100d.txt.bz2"
with open(newfilepath, 'wb') as new_file, bz2.BZ2File(filepath, 'rb') as file:
    for data in iter(lambda : file.read(100 * 1024), b''):
        new_file.write(data)

In [ ]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(newfilepath)

# Similaridade de Documentos

Para realizar a similaridade entre documentos, utilize as frases abaixo:

In [ ]:
frase1 = "Excelente produto chegou antes do prazo indico e recomendo produto bom pois já testei e foi mais que aprovado"
frase2 = "SUPER RECOMENDO, PREÇO, QUALIDADE #BRASTEMP, EFICIÊNCIA NA ENTREGA, E FACILIDADE DE PAGAMENTO. MUITO BOM!!!"
frase3 = "A tampa do fogão veio com problemas com o pino de encaixe solto e precisa de reparos"
frase4 = "Fogão ótimo!"

## Distância de Jaccard

<b> Atividade </b>

1) Faça um método que calcule a similaridade de Jaccard e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: lembrando que você precisa aplicar um pre-processamento nessas frases antes de aplicar o método.
Faça:

- Lower
- Remoção StopWords
- Remoção Pontuação
- Tokenização

In [ ]:
def pre_processamento_texto(corpus):

    #token
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)

    #aplicar lower
    corpus_alt = [t.lower() for t in corpus_alt]

    #remove stops
    #portuguese_stops = stopwords.words('portuguese')
    #corpus_alt = [p for p in corpus_alt if p not in portuguese_stops]

    #remove numeros
    corpus_alt = [re.sub(r'\d', '', p) for p in corpus_alt]

    #remove pontuacoes
    corpus_alt = [p for p in corpus_alt if p not in string.punctuation]

    #remove acentos
    corpus_alt = [unidecode(p) for p in corpus_alt]

    return corpus_alt

In [ ]:
frase1_preprocessada = pre_processamento_texto(frase1)
frase2_preprocessada = pre_processamento_texto(frase2)
frase3_preprocessada = pre_processamento_texto(frase3)
frase4_preprocessada = pre_processamento_texto(frase4)

In [ ]:
frase1_preprocessada

['excelente',
 'produto',
 'chegou',
 'antes',
 'do',
 'prazo',
 'indico',
 'e',
 'recomendo',
 'produto',
 'bom',
 'pois',
 'ja',
 'testei',
 'e',
 'foi',
 'mais',
 'que',
 'aprovado']

In [ ]:
def jaccard(f1, f2):
    return len(set(f1).intersection(set(f2)))/len(set(f1).union(set(f2)))

In [ ]:
print('Frase 1 e 2 = ', jaccard(frase1_preprocessada, frase2_preprocessada))
print('Frase 1 e 3 = ', jaccard(frase1_preprocessada, frase3_preprocessada))
print('Frase 2 e 3 = ', jaccard(frase2_preprocessada, frase3_preprocessada))
print('Frase 1 e 4 = ', jaccard(frase1_preprocessada, frase4_preprocessada))

Frase 1 e 2 =  0.10714285714285714
Frase 1 e 3 =  0.06666666666666667
Frase 2 e 3 =  0.07407407407407407
Frase 1 e 4 =  0.0


<b> Atividade </b>

2) Qual par de frase teve maior simlaridade? E qual teve menor? Este resultado faz sentido? Explique.



---



A maior similaridade foi entre as frases 1 e 2 com o valor de: 0.10714285714285714
A menor similaridade foi entre as frases 1 e 4 com valor 0
Faz sentido pois na frase 4 apenas foi dito Fogão ótimo, nas outras frases tiveram mais detalhes e mais chance de serem similares

## Distância de Cosseno

Aqui iremos calcular a distância do cosseno utilizando duas formas, que aprendemos na aula passada, para representar o texto.

- Bag of Words (BOW)
- Embedding

Observação:

Existem duas formas de trabalhar com o cosseno:

<b> Distância </b>: quanto menor mais perto estão as frases.
<b> Similaridade </b>: quanto maior mais perto estão as frases.

### BOW - Distância do cosseno

<b> Atividade </b>

3) Calcule a distância do cosseno utilizando a representação CountVectorizer e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: no CountVectorizer utilizem as frases já pre-processadas da atividade anterior. Mas para aplicá-las no fit_transform, cada frase deve ser um string (sem estar tokenizada) dentro de uma lista.

```python
#exemplo
distance.cosine(frase1, frase2)
```

In [ ]:
vetor = CountVectorizer()

In [ ]:
frases_bow = vetor.fit_transform([" ".join(frase1_preprocessada),
                     " ".join(frase2_preprocessada),
                     " ".join(frase3_preprocessada),
                     " ".join(frase4_preprocessada),])

In [ ]:
frases_bow.todense()

matrix([[1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 1, 1, 0, 0, 0, 2, 0, 1, 1, 0, 0, 0, 0, 1, 0],
        [0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
         1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 2, 2, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [ ]:
frase1_bow = frases_bow.todense()[0]
frase2_bow = frases_bow.todense()[1]
frase3_bow = frases_bow.todense()[2]
frase4_bow = frases_bow.todense()[3]

In [ ]:
print(distance.cosine(frase1_bow, frase2_bow))
print(distance.cosine(frase1_bow, frase3_bow))
print(distance.cosine(frase2_bow, frase3_bow))
print(distance.cosine(frase1_bow, frase4_bow))

0.8727430474048444
0.9459261929564124
0.8692559099078774
1.0


<b> Atividade </b>


4) Qual par de frase teve maior distância? E qual teve menor? Este resultado faz sentido? Explique.

Maior distância entre as frases 1 e 4
Menor distância entre as frases 2 e 3
Faz sentido pois quanto menor a distância maior a similaridade entre as frases

### Embedding - Distância do cosseno

Para calcular o embedding de cada uma das frases, utilize o modelo carregado inicialmente.

Cada palavra tem um vetor, para formar o embedding da frase tire a média de todos os vetores.

Utilize as frases já pre-processadas

<b> Atividade </b>

5) Calcule a distância do cosseno utilizando a representação Embedding e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4


In [ ]:
def get_word2vec(frase):
    return np.mean(np.array([word_vectors[p] for p in frase if p in word_vectors.vocab]), axis=0)

In [ ]:
frase1_emb = get_word2vec(frase1_preprocessada)
frase2_emb = get_word2vec(frase2_preprocessada)
frase3_emb = get_word2vec(frase3_preprocessada)
frase4_emb = get_word2vec(frase4_preprocessada)

In [ ]:
print(distance.cosine(frase1_emb, frase2_emb))
print(distance.cosine(frase1_emb, frase3_emb))
print(distance.cosine(frase2_emb, frase3_emb))
print(distance.cosine(frase1_emb, frase4_emb))

0.10306441783905029
0.13053971529006958
0.143882155418396
0.35638290643692017


<b>Atividade </b>

6) Qual par de frase teve maior distância? E qual teve menor? Este resultado faz sentido? Explique.

Maior distância entre as frases 1 e 4
Menor distância entre as frases 1 e 2
Faz sentido pois quanto maior a distância menor a similaridade, contudo percebe-se que percentualmente o modelo embending teve valores diferentes do bag of words

## WMD

O WMD já está incorporado ao Word2Vec

<b> Atividade </b>

7) Calcule a distância WMD e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: use a variável já tokenizada.

In [ ]:
from pyemd import emd
from gensim.similarities import WmdSimilarity
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedLineDocument

In [ ]:
print(word_vectors.wmdistance(frase1_preprocessada, frase2_preprocessada))
print(word_vectors.wmdistance(frase1_preprocessada, frase3_preprocessada))
print(word_vectors.wmdistance(frase2_preprocessada, frase3_preprocessada))
print(word_vectors.wmdistance(frase1_preprocessada, frase4_preprocessada))

2.6522280051386518
2.8301634750747398
2.8800446266377127
4.148723259169023


<b> Atividade </b>

8) Qual par de frase teve maior distância? E qual teve menor? Este resultado faz sentido? Explique.

Maior distância entre as frases 1 e 4
Menor distância entre as frases 1 e 2
Novamente faz sentido o resultado já era esperado maior distância entre as frases 1 e 4
Contudo novamente percebe-se uma diferença entre os métodos, que deve ser analisado de acordo com o tamanho do dataset e objetivo qual deve ser utilizado além da quantidade de features

# Classificação de Documentos

A clssificação de documentos é muito útil em vários aspectos. Um dos tipos de classificação de texto é a análise de sentimentos.

A fim de ilustrar a classificação de documentos iremos criar um modelo para classificar uma frase como positiva ou negativa.

## Dataset

<b> Atividade </b>

9) Carregue o dataset com o pandas e depois dê o head no dataframe.


Link download: https://drive.google.com/open?id=15azJWdEEPGsXQGiDmEOseTBJcquWvBQc

<b> Este dataset é sobre revisões de filmes do IMDB. </b>

In [ ]:
df = pd.read_csv("imdb-reviews-pt-br.csv")

In [ ]:
df.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [ ]:
df['sentiment'].value_counts()

neg    24765
pos    24694
Name: sentiment, dtype: int64

## Representação dos dados

O sentimento positivo e negativo iremos binarizar cada um deles. Seja 1 positivo e 0 negativo.

Iremos representar o texto de duas formas:

- Bag of Words (BOW)
- Embedding

Depois iremos comparar o resultado de cada um deles.

### Representação Target

<b> Atividade </b>

10) Faça a representação dos sentimentos. 1 positivo; 0 negativo

In [ ]:
target = df['sentiment'].replace(['neg', 'pos'], [0, 1])
target

0        0
1        0
2        0
3        0
4        0
        ..
49454    1
49455    1
49456    1
49457    1
49458    1
Name: sentiment, Length: 49459, dtype: int64

### Bag of Words (BOW)

<b> Atividade </b>

11) Aplique o pré-processamento listado abaixo na coluna ``text_pt`` (crie uma nova coluna ```text_pt_sem_stopwords``` no dataframe para armazenar este dado processado):

- Remova as stopwords do texto
- Remova as pontuções
- Mantenha o texto sem tokenização, ou seja uma string

<b> Dica: </b> use o ```progress_apply``` para exibir a barra de progresso:

```python
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
df["colunas"].progress_apply(lambda x: preprocessamento(x))
```

In [ ]:
def pre_processamento_texto_retorna_string(corpus):

    #token
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)

    #aplicar lower
    corpus_alt = [t.lower() for t in corpus_alt]

    #remove stops
    portuguese_stops = stopwords.words('portuguese')
    corpus_alt = [p for p in corpus_alt if p not in portuguese_stops]

    #remove numeros
    corpus_alt = [re.sub(r'\d', '', p) for p in corpus_alt]

    #remove pontuacoes
    corpus_alt = [p for p in corpus_alt if p not in string.punctuation]

    #remove acentos
    corpus_alt = [unidecode(p) for p in corpus_alt]

    corpus_alt = ' '.join(corpus_alt)

    return corpus_alt

In [ ]:
df['texto_pt_limpo'] = df['text_pt'].apply(lambda x : pre_processamento_texto_retorna_string(x))

In [ ]:
df.head()

,id,text_en,text_pt,sentiment,texto_pt_limpo
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,vez sr costner arrumou filme tempo necessario ...
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,exemplo motivo maioria filmes acao mesmos gene...
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,primeiro tudo odeio raps imbecis poderiam agir...
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,beatles puderam escrever musicas todos gostass...
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,filmes fotos latao palavra apropriada verdade ...


<b> Atividade </b>

12) Aplique a representação do texto processado anteriormente com CountVectorizer.

In [ ]:
vector = CountVectorizer()

In [ ]:
X_bow = vetor.fit_transform(df['texto_pt_limpo'])

In [ ]:
X_bow

<49459x125217 sparse matrix of type '<class 'numpy.int64'>'
	with 5028654 stored elements in Compressed Sparse Row format>

### Embedding

<b> Atividade </b>

13) Aplique o pré-processamento listado abaixo na coluna ``text_pt`` (crie uma nova coluna ```text_pt_sem_stopwords_token``` no dataframe para armazenar este dado processado):

- Aplique lower
- Remova as stopwords do texto
- Remova as pontuções
- Mantenha o texto com tokenização

<b> Dica: </b> use o ```progress_apply``` para exibir a barra de progresso:

```python
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
df["colunas"].progress_apply(lambda x: preprocessamento(x))
```

In [ ]:
def pre_processamento_texto_retorna_token(corpus):

    #token
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)

    #aplicar lower
    corpus_alt = [t.lower() for t in corpus_alt]

    #remove stops
    portuguese_stops = stopwords.words('portuguese')
    corpus_alt = [p for p in corpus_alt if p not in portuguese_stops]

    #remove numeros
    corpus_alt = [re.sub(r'\d', '', p) for p in corpus_alt]

    #remove pontuacoes
    corpus_alt = [p for p in corpus_alt if p not in string.punctuation]

    #remove acentos
    corpus_alt = [unidecode(p) for p in corpus_alt]


    return corpus_alt

In [ ]:
df['texto_pt_limpo_token'] = df['text_pt'].apply(lambda x : pre_processamento_texto_retorna_token(x))

In [ ]:
df['texto_pt_limpo_token']

0        [vez, sr, costner, arrumou, filme, tempo, nece...
1        [exemplo, motivo, maioria, filmes, acao, mesmo...
2        [primeiro, tudo, odeio, raps, imbecis, poderia...
3        [beatles, puderam, escrever, musicas, todos, g...
4        [filmes, fotos, latao, palavra, apropriada, ve...
                               ...                        
49454    [media, votos, baixa, fato, funcionario, locad...
49455    [enredo, algumas, reviravoltas, infelizes, ina...
49456    [espantado, forma, filme, maioria, outros, med...
49457    [christmas, together, realmente, veio, antes, ...
49458    [drama, romantico, classe, trabalhadora, diret...
Name: texto_pt_limpo_token, Length: 49459, dtype: object


<b> Atividade </b>

14) Aplique a representação do texto com Embeddings. Cada palavra tem um embedding, o embedding da frase é a média de todos embeddings.

In [ ]:
def get_word2vec(frase):
    return np.mean(np.array([word_vectors[p] for p in frase if p in word_vectors.vocab]), axis=0)

In [ ]:
X_word2vec = df['texto_pt_limpo_token'].apply(lambda x : get_word2vec(x))

In [ ]:
X_word2vec

0        [0.15084998, 0.19326542, -0.056642305, -0.2193...
1        [0.1662331, 0.22085148, -0.066288956, -0.20869...
2        [0.22359823, 0.1167883, -0.01743333, -0.199178...
3        [0.2631776, 0.12153541, -0.07142547, -0.212037...
4        [0.21868937, 0.09785001, -0.07797347, -0.15642...
                               ...                        
49454    [0.17877448, 0.14892156, -0.08598039, -0.10364...
49455    [0.17312221, 0.13372223, -0.026474074, -0.1776...
49456    [0.18755417, 0.17317419, -0.056084007, -0.1863...
49457    [0.18442088, 0.14988786, -0.10535385, -0.21823...
49458    [0.27497926, 0.17049538, -0.053712655, -0.1587...
Name: texto_pt_limpo_token, Length: 49459, dtype: object

## Treinamento

### CountVectorizer


<b> Atividade </b>

15) Faça a divisão dados dados em treino e teste como no exemplo abaixo:

```python
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bag, target,random_state=123)
```

In [ ]:
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bow, target, random_state=123)


<b> Atividade </b>

16) Treine com uma regressão logística

In [ ]:
modelo_bow = LogisticRegression()

In [ ]:
modelo_bow.fit(X_train_bow, y_train_bow)

LogisticRegression()


<b> Atividade </b>

17) Calcule as métricas de resultado utilizando método abaixo:

```python
print(classification_report(y_test_bow, y_pred))
```

In [ ]:
y_pred = modelo_bow.predict(X_test_bow)

In [ ]:
print(classification_report(y_test_bow, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.87      0.88      6112
           1       0.87      0.89      0.88      6253

    accuracy                           0.88     12365
   macro avg       0.88      0.88      0.88     12365
weighted avg       0.88      0.88      0.88     12365



### Embedding


<b> Atividade </b>

18) Faça a divisão dados dados em treino e teste como no exemplo abaixo:

Verifique o shape do X treino e X teste. Caso eles estejam com apenas uma dimensão, você precisa tranformá-los para duas dimensões, caso contrário ocorrerá erro no treinamento.

In [ ]:
X_train_embedding, X_test_embedding, y_train_embedding, y_test_embedding = \
train_test_split(X_word2vec.values, target, random_state=123)


<b> Atividade </b>

19) Treine com uma regressão logística

In [ ]:
modelo_embeding = LogisticRegression()

In [ ]:
X_train_embedding = pd.DataFrame([x for x in X_train_embedding])
X_test_embedding = pd.DataFrame([x for x in X_test_embedding])

In [ ]:
modelo_embeding.fit(X_train_embedding, y_train_embedding)

LogisticRegression()


<b> Atividade </b>

20) Calcule as métricas de resultado utilizando método abaixo:

```python
print(classification_report(y_test_bow, y_pred))
```

#### Calcule as métricas

In [ ]:
y_pred = modelo_embeding.predict(X_test_embedding)

In [ ]:
print(classification_report(y_test_embedding, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.76      0.76      6112
           1       0.77      0.76      0.76      6253

    accuracy                           0.76     12365
   macro avg       0.76      0.76      0.76     12365
weighted avg       0.76      0.76      0.76     12365




<b> Atividade </b>

21) Compare os resultados obtidos com o BagOfWords e com o Embedding. Explique os possíveis motivos desta diferença.

Nesse caso acima tivemos o melhor resultado no modelo treinado usando BOW devido ao shape do dataset, o que nesse caso BOW tinha mais features e impactou num resultado melhor, porém nem sempre isso acontece deve-se atentar para riscos de overfinting.

# Análise de sentimentos

O modelo que criamos anteriormente é para ilustrar como podemos realizar classificação de documentos.
Quando a tarefa é sobre análise de sentimentos, temos duas opções: treinar nosso próprio modelo, como feito anteriormente ou utilizar uma das inúmeras ferramentas prontas.

Vamos testar as seguintes ferramentas:

- Vader
- Textblob
- Affin

Nesta atividade iremos utilizar as duas variáveis abaixo:

In [ ]:
texto_neg = df.loc[0, "text_en"]
texto_pos = df.loc[49431, "text_en"]

## Vader

<b> Apenas Inglês </b>

O VADER (Valence Aware Dictionary e sEntiment Reasoner) é uma ferramenta de análise de sentimentos baseada em regras e léxico, especificamente identifica os sentimentos expressos nas mídias sociais.

- positive sentiment: compound score >= 0.05
- neutral sentiment: (compound score > -0.05) e (compound score < 0.05)
- negative sentiment: compound score <= -0.05

Mais informações: https://github.com/cjhutto/vaderSentiment


<b> Atividade </b>

22) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(texto)
```

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(texto_pos)

{'neg': 0.084, 'neu': 0.737, 'pos': 0.179, 'compound': 0.9969}

In [ ]:
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(texto_neg)

{'neg': 0.126, 'neu': 0.76, 'pos': 0.114, 'compound': 0.3958}

## TextBlob

<b> Apenas inglês </b>

https://www.presentslide.in/2019/08/sentiment-analysis-textblob-library.html

<b> Atividade </b>

23) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
sentence=TextBlob(texto)
sentence.sentiment
```

In [ ]:
from textblob import TextBlob

In [ ]:
sentence=TextBlob(texto_pos)
sentence.sentiment

Sentiment(polarity=0.190819118692253, subjectivity=0.6026226012793177)

In [ ]:
sentence=TextBlob(texto_neg)
sentence.sentiment

Sentiment(polarity=0.06385964912280702, subjectivity=0.5629824561403508)

## Afinn

- Valor maior que 0 indica sentimento positivo
- Valor menor que 0 indica sentimento negativo

<b> Atividade </b>

24) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
afinn = Afinn()
afinn.score(texto)
```

In [ ]:
from afinn import Afinn

In [ ]:
afinn = Afinn()
afinn.score(texto_pos)

55.0

In [ ]:
afinn = Afinn()
afinn.score(texto_neg)

8.0

<b> Atividade </b>

25) Para você, qual ferramenta teve melhor comportamento?

O Vader contribuiu bem na análise de texto positivo destacando maiores pesos em sentimento positivo e neutro, contudo não contribuiu para análise de texto negativo, ficando maior peso no neutro.

O Textblob apresentou polaridade maior para o text_pos fortalecendo um sentimento mais positivo para o mesmo, porém a subjectividade ficou bem próxima nos dois textos, não sendo nesse caso a ferramenta mais indicada para análise.

O Affin no meu ponto de vista fez mais sentido pois considerando que maior que zero indica sentimento positivo temos ambos os textos maior que zero, contudo o score do texto positivo é bem maior que o score do texto negativo, apontando assim um sentimento mais positivo no texto_pos .

# Dica:
## Quando for trabalhar com um dataset em inglês, a biblioteca Spacy facilita!

In [ ]:
!python -m spacy download en_core_web_md
#!pip install -U spacy

C:\Users\user\AppData\Roaming\Python36\python.exe: No module named spacy


In [ ]:
import spacy
import pandas as pd

O scpay forne um pacote que já tem série de modelos já treinados em NLP. Inclusive para os embeddings em inglês.

Para mais informações vá em:

https://spacy.io/models/en#en_core_web_md



Com o método abaixo carregamos um dos modelos do spacy:

```python
nlp = spacy.load('en_core_web_md')
```

Para aplicar o modelo, basta passar o texto para o modelo carregado anteriormente:

```python
doc = nlp("This is some text that I am processing with Spacy")
```

Carregue o modelo e imprima doc

In [ ]:
nlp = spacy.load('en_core_web_md')

In [ ]:
doc = nlp("This is some text that I am processing with Spacy")

In [ ]:
doc

Ao aplicar o modelo carregado a variável <b> doc </b> já possui os embeddings de cada uma das palavras e o embedding da frase, que é a média de todos vetores de todas palavras.

```python
#vetor da primeira palavra
doc[0].vector
#vetor agregado pela média - embedding do documento
doc.vector
```
O código abaixo mostra que a média de uma posição em específico dos embeddings de todas as palavras e a posição do embedding do documento possuem o mesmo valor.

In [ ]:
def calcula_media_posicao(x):
    soma = 0
    vector = []
    for i in range(0,len(doc)):
        vector.append(doc[i].vector)

    for v in vector:
        soma += v[x]
    return soma/len(doc)

In [ ]:
round(calcula_media_posicao(10),6)

In [ ]:
round(doc.vector[10], 6)